# VacationPy

In [1]:
# import dependencies
import gmaps
import pandas as pd
import requests
import json 

# import google api key
from config import google_api_key

# access maps with unique api key
gmaps.configure(api_key = google_api_key)

In [2]:
# read in data from part i
cities = pd.read_csv("../data/cities.csv")

# preview dataset
cities.head()

,City,Country,Date,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness
0,Punta Arenas,CL,1579928831,-53.15,-70.92,48.20,87,12.75,20
1,Cloquet,US,1579928831,46.72,-92.46,32.00,79,10.29,100
2,Traverse City,US,1579928831,44.76,-85.62,34.00,86,4.70,90
3,Hofn,IS,1579928635,64.25,-15.21,35.42,97,52.61,100
4,Cairns,AU,1579928635,-16.92,145.77,89.01,74,3.36,61


## Create Humidity Heatmap

In [3]:
# store locations and humidities
locations = cities[["Latitude", "Longitude"]].astype(float)
humidities = cities["Humidity"].astype(float)

In [4]:
# create figure zoomed in and centered at coordinates (0, 0)
fig = gmaps.figure(zoom_level = 2, center = (0,0))

# create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = humidities, 
                                 dissipating = False, 
                                 max_intensity = humidities.max(), 
                                 point_radius = 2)

# apply heatmap layer to map
fig.add_layer(heat_layer)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))

## Find Ideal Weather Conditions
* Temperature between 60 and 80 degrees Farenheit
* Humidity less than or equal to 50%
* Cloudiness less than or equal to 25%
* Wind Speed less than or equal to 10 MPH

In [5]:
# temperature between 60 and 80 degrees
ideal_temp = (cities["Max Temp"] >= 60) & (cities["Max Temp"] <= 80)

# humidity lower than 50%
ideal_humid = cities["Humidity"] <= 50

# cloudiness lower than 25%
ideal_cloud = cities["Cloudiness"] <= 25

# wind speed less than 10 mph
ideal_wind = cities["Wind Speed"] <= 10

# filter cities on above conditions
cities_ideal = cities.loc[ideal_temp & ideal_humid & ideal_cloud & ideal_wind]

# add column to store name of nearest hotel
cities_ideal["Nearest Hotel"] = ""

/Users/speculadora/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Perform API Calls

In [6]:
# build url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# loop through rows in cities_ideal and hit api for nearest hotel
for index, row in cities_ideal.iterrows():
    
    # store city of current iteration
    current_city = row.get("City")
    
    # store query parameters
    params = {"location": str(row.get("Latitude")) + "," + str(row.get("Longitude")),
              "rankby": "distance",
              "type": "lodging",
              "keyword": "hotel",
              "key": google_api_key}
        
    # assemble url and make api request
    print(f"Retrieving results for {current_city}")
    response = requests.get(base_url, params = params).json()
    
    # attempt to store data
    try:
        hotel = response["results"][0]["name"]
        print(f"Closest hotel to {current_city}: {hotel}")
        cities_ideal["Nearest Hotel"][index] = hotel
    
    # return a message if there is an error
    except (KeyError, IndexError):
        print(f"Could not find a hotel within 5000m of {current_city}")
        
    print("------------------------------------")

Retrieving results for Pokaran
Closest hotel to Pokaran: HOTEL SRI SAI PALACE
------------------------------------
Retrieving results for Dwarka


/Users/speculadora/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/speculadora/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


Closest hotel to Dwarka: Hotel Shree Vallabh
------------------------------------
Retrieving results for Tura
Closest hotel to Tura: Hotel RIKMAN Continental
------------------------------------
Retrieving results for Anupgarh
Closest hotel to Anupgarh: SRK Hotel
------------------------------------
Retrieving results for Botro
Closest hotel to Botro: Hotel Akpole
------------------------------------
Retrieving results for Opuwo
Closest hotel to Opuwo: Ohakane Guesthouse
------------------------------------


## Plot Hotels on Humidity Heatmap

In [7]:
# create template for tooltip
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# for each city, store relevant data in a dictionary, append the dictionary to a list
city_info = []
for index, row in cities_ideal.iterrows():
    city_dict = {}
    city_dict["Nearest Hotel"] = row.get("Nearest Hotel")
    city_dict["City"] = row.get("City")
    city_dict["Country"] = row.get("Country")
    city_info.append(city_dict)
    

# list comprehension to format city information for tooltip
city_info_boxes = [info_box_template.format(**city) for city in city_info]

# store city locations
city_locations = cities_ideal[["Latitude", "Longitude"]]

# create marker layer
marker_layer = gmaps.marker_layer(city_locations, info_box_content = city_info_boxes)

In [8]:
# create a new map figure zoomed & centered at (0,0)
fig = gmaps.figure(zoom_level = 2, center = (0,0))

# add heatmap and marker layers
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))